<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод:" data-toc-modified-id="Общий-вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод:</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, а также обоснуем корректность его работы.

*Цель*: защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

*План действий*:
- загрузим и изучим данные
- выдвенем гипотезу о способе преобразований
- докажем ее математически
- сформулируем общий алгортим действий для преобразований 
- протестируем алгоритм на входных данных

## Загрузка данных

In [1]:
#импортируем все нужные библиотеки и функции
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Входные данные представленны таблицей в которой каждая строчка представляет собой личные данные одного клиента банка.

- *Признаки*: пол, возраст и зарплата застрахованного, количество членов его семьи.
- *Целевой признак*: количество страховых выплат клиенту за последние 5 лет.


## Умножение матриц

Сначала запишем известные формулы для линенйно регрессии с пояснениями.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

*Предсказания*:

$$
a = Xw   
$$

*Задача обучения*:

$$
w = \arg\min_w MSE(Xw, y)
$$

*Формула обучения*:

$$
w = (X^T X)^{-1} X^T y
$$

-------------------------------------------------------------------------------------------------------------------------------

**Сформулируем гипотезу**: умножение матрицы признаков на произвольную обратимую матрицу не изменит качества линейной регрессии.

Попробуем подтвердить ее.


**Цель** -  доказать что  

$$
a_{new} = a
$$

Для начала посмотрим как изменит умнжение на обратную матрицу формулу обучения. Домножим каждую матрицу **X** на обратную **P**.

$$
( (XP)^T \cdot (XP) )^{-1} \cdot (X P)^T \cdot y
$$

С помощью свойств транспонирования и обратной матрицы упростим формулу.

$$
(P^T\cdot  X^T\cdot  X\cdot  P)^{-1} \cdot P^T \cdot X^T \cdot y
$$

$$
(P^T\cdot (X^T  X)\cdot  P)^{-1} \cdot P^T \cdot X^T \cdot y
$$

$$
P^{-1} \cdot (X^T  X)^{-1} \cdot (P^T)^{-1} \cdot P^T \cdot X^T \cdot y
$$


Здесь наблюдаем произведение обратной и обратимой матриц, которые можно просто сократить, так как умножение на единичную матрицу ничего не меняет.

$$
P^{-1} \cdot (X^T X)^{-1} \cdot X^T \cdot y
$$

Правая часть совпадает с формулой обучения, а значит

$$
w_{new} = P^{-1} w
$$

Теперь зная, как изменится формула предсказания для модели с преобразованными признаками

$$
a_{new} = XP w_{new}
$$

$$
a_{new} = XPP^{-1} w
$$

Посередине явно получается единичная матрица от которой избавляемся и получаем, что

$$
a_{new} = X w = a
$$

### Вывод:

При использовании модели линейной регрессии, можно умножить матрицу признаков на произвольную обратимую матрицу. Если генерирорвать такую обратимую случайно, то персональные данные клиентов будут под защитой.

## Алгоритм преобразования

Теперь опишем полный алгоритм преобразования.

Так как основная идея заключается в умножении обучающих признаков на случайную обратимую матрицу, то напишем отдельную функцию для такого преобразования признаков.

In [3]:
def transform_features_invmatrix(features):
    '''
    Функция принимает в себя признаки в формате DataFrame
    Возвращает преобразованные признаки:
    Случайная квадратная обратимая матрица такого же размера сколько и колонок в признаках умнажается на матрицу признаков
    '''
    size = features.shape[1]
    
    while True:
        inv_matrix = np.random.normal(size=(size,size))
        try:
            np.linalg.inv(inv_matrix)
            break
        except:
            continue
    features_up = (features.values).dot(inv_matrix)
    
    return pd.DataFrame(features_up, columns=features.columns, index=features.index) 

Получив искомую матрицу алгоритм будет формулирувоаться следующим образом:

- отделить обучающие признаки от целевого
- преобразовать их путем умножения на произвольную случайную матрицу (т.е. воспользоваться созданной функцией)
- обучить модель линейной регрессии на преобразованных признаках

## Проверка алгоритма

Теперь проверим как работает алгоритм на предоставленных данных.

Сначала узнаем результаты работы на признаках без преобразований.

In [4]:
#разделяем на целевой и обучающие признаки, а также на обучающую и тренироваочную выборки
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=5)

In [5]:
#обучаем модель и выводим результат метрки R2
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)

score1 = r2_score(target_test, predictions)
print('Резульат без преобразований:', score1)

Резульат без преобразований: 0.4293735984075683


Теперь преобразуем признаки и обучим новую модель на них.

In [6]:
features_up = transform_features_invmatrix(features)

features_up_train, features_up_test = train_test_split(features_up, test_size=0.25, random_state=5)

In [7]:
model_up = LinearRegression()
model_up.fit(features_up_train, target_train)
predictions_up = model_up.predict(features_up_test)

score2 = r2_score(target_test, predictions_up)
print('Резульат c преобразованиями:', score2)

Резульат c преобразованиями: 0.42937359840732137


Результаты почти идентичны, проверим насколько они отличатся.

In [8]:
#проверка на идентичность
score1 - score2

2.469136006766348e-13

Отличие идет только после 14 знака в дробной части. Подобное можно списать на округление при вычислениях. Соответственно результат подтвердил правильность алгоритма.

## Общий вывод:

Входные данные представляли собой таблицу, в которой каждая строка являлась набором личных данных клиентов банка.

Нужно было защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

Была выдвинута *гипотеза*, что цель достигнется при умножении матрицы обучающих признаков на произвольную обратимую матрицу.
- Гипотеза была доказана математически - путем преобразования формул обучения и предсказания линенйной регрессии.
- Затем был сформирован алгоритм и на его основе мы осуществили преобразование входных данных.
- Сравнение результатов по метрике качества R2 показали идентичные результаты для предсказания по преобразованным признакам и признакам без преобразований, что эмпирически еше раз подтвердило гипотезу.